In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# FP-growth 
上一章提到Apriori利用frequent itemsets和关联法则来观察数据里一些有趣的东西，这一章讲的FP-growth算法是一种分治算法，只关注frequent itemsets，但大大的提高了速度，简单来说，FP-growth算法由两步所构成
1. 建立FP树
2. 从FP树挖掘frequent itemsets

FP树的nodes之间代表关联性，nodes也会在同一棵树重复出现，建立FP树的流程如下
1. 计算个别item的频次(support)从大到小排序，个别node代表<item:count>
2. 用排序好的记录构成前缀树T，当树只剩下一个分支的时候，生成频率模式(header table)
3. 用BFS的方式遍历，对每个item生成条件header table
4. 重复1-3  

可参考https://zhuanlan.zhihu.com/p/50339839

### 优缺点

优点：比Apriori还快
缺点：较难复现、特定数据会使得速度变慢


接下来我们定义FP树的数据结构

In [6]:
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode      #needs to be updated
        self.children = {} 
    
    def increment(self, numOccur):
        self.count += numOccur
        
    def display(self, ind=1):
        print('  '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.display(ind+1)

In [8]:
rootNode=treeNode('pyramid',9,None)
rootNode.children['eye']=treeNode('eye',13,None)
rootNode.children['phoenix']=treeNode('phoenix',3,None)
rootNode.display()

   pyramid   9
     eye   13
     phoenix   3


接下来介绍如何建立FP-tree

In [17]:
def createTree(dataSet, minSup=1): #create FP-tree from dataset but don't mine
    headerTable = {}
    #go over dataSet twice
    for trans in dataSet:#first pass counts frequency of occurance
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    ks=list(headerTable.keys()).copy()
    for k in ks:  #remove items not meeting minSup
        if headerTable[k] < minSup: 
            del(headerTable[k])
    freqItemSet = set(headerTable.keys())
    #print 'freqItemSet: ',freqItemSet
    if len(freqItemSet) == 0: return None, None  #if no items meet min support -->get out
    for k in headerTable:
        headerTable[k] = [headerTable[k], None] #reformat headerTable to use Node link 
    #print 'headerTable: ',headerTable
    retTree = treeNode('Null Set', 1, None) #create tree
    for tranSet, count in dataSet.items():  #go through dataset 2nd time
        localD = {}
        for item in tranSet:  #put transaction items in order
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
            updateTree(orderedItems, retTree, headerTable, count)#populate tree with ordered freq itemset
    return retTree, headerTable #return tree and header table

def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:#check if orderedItems[0] in retTree.children
        inTree.children[items[0]].increment(count) #incrament count
    else:   #add items[0] to inTree.children
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] == None: #update header table 
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    if len(items) > 1:#call updateTree() with remaining ordered items
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)
        
def updateHeader(nodeToTest, targetNode):   #this version does not use recursion
    while (nodeToTest.nodeLink != None):    #Do not use recursion to traverse a linked list!
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

simpleData=loadSimpDat()
initSet=createInitSet(simpleData)
initSet

{frozenset({'z'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'h', 'j', 'p', 'r', 'z'}): 1,
 frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1}

In [44]:
FPTree, header=createTree(initSet,3)
FPTree.display()
print(header['x'][1].count)
sorted(header.items(), key=lambda p: p[1][0])

   Null Set   1
     z   5
       x   3
         t   2
           y   2
             s   2
         y   1
           t   1
             r   1
       r   1
     x   1
       r   1
         s   1
3


[('y', [3, <__main__.treeNode at 0x7f1e55349d30>]),
 ('s', [3, <__main__.treeNode at 0x7f1e55349e80>]),
 ('t', [3, <__main__.treeNode at 0x7f1e55349d68>]),
 ('r', [3, <__main__.treeNode at 0x7f1e55349da0>]),
 ('x', [4, <__main__.treeNode at 0x7f1e55349cf8>]),
 ('z', [5, <__main__.treeNode at 0x7f1e55349cc0>])]

接下来，从建立好的FPTree来建立条件header table

In [22]:
def ascendTree(leafNode, prefixPath): #ascends from leaf node to root
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)
    
def findPrefixPath(basePat, treeNode): #treeNode comes from header table
    condPats = {}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1: 
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

findPrefixPath('x', header['x'][1]), findPrefixPath('z', header['z'][1]), findPrefixPath('r', header['r'][1])

({frozenset({'z'}): 3},
 {},
 {frozenset({'x'}): 1,
  frozenset({'z'}): 1,
  frozenset({'t', 'x', 'y', 'z'}): 1})

In [47]:
def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: p[1][0])]#(sort header table)
    for basePat in bigL:  #start from bottom of header table
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        #print 'finalFrequent Item: ',newFreqSet    #append to set
        freqItemList.append(newFreqSet)
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1])
        #print 'condPattBases :',basePat, condPattBases
        #2. construct cond FP-tree from cond. pattern base
        myCondTree, myHead = createTree(condPattBases, minSup)
        #print 'head from conditional tree: ', myHead
        if myHead != None: #3. mine cond. FP-tree
            print('conditional tree for: ',newFreqSet)
            myCondTree.display(1)            
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)

freqItems=[]
mineTree(FPTree,header,3,set([]),freqItems)

conditional tree for:  {'y'}
   Null Set   1
     z   3
       x   3
conditional tree for:  {'x', 'y'}
   Null Set   1
     z   3
conditional tree for:  {'s'}
   Null Set   1
     x   3
conditional tree for:  {'t'}
   Null Set   1
     z   3
       x   3
conditional tree for:  {'x', 't'}
   Null Set   1
     z   3
conditional tree for:  {'x'}
   Null Set   1
     z   3


In [48]:
freqItems

[{'y'},
 {'y', 'z'},
 {'x', 'y'},
 {'x', 'y', 'z'},
 {'s'},
 {'s', 'x'},
 {'t'},
 {'t', 'z'},
 {'t', 'x'},
 {'t', 'x', 'z'},
 {'r'},
 {'x'},
 {'x', 'z'},
 {'z'}]

接下来这个数据有100万个例子，可能会跑比较久，每一行代表词向量，找出同时频繁出现的字词组合

In [59]:
parsedData=[line.split() for line in open('./datasets/kosarak.dat').readlines()]
initSet=createInitSet(parsedData)
fpTree, header=createTree(initSet,10000)
freqList=[]
mineTree(fpTree,header,100000,set([]),freqList)
len(freqList)

conditional tree for:  {'1'}
   Null Set   1
     6   107404
conditional tree for:  {'3'}
   Null Set   1
     11   9718
     6   186289
       11   117401
conditional tree for:  {'11', '3'}
   Null Set   1
     6   117401
conditional tree for:  {'11'}
   Null Set   1
     6   261773


54

In [60]:
freqList

[{'28'},
 {'423'},
 {'378'},
 {'32'},
 {'14'},
 {'155'},
 {'364'},
 {'49'},
 {'25'},
 {'512'},
 {'504'},
 {'73'},
 {'314'},
 {'897'},
 {'56'},
 {'273'},
 {'737'},
 {'269'},
 {'490'},
 {'987'},
 {'135'},
 {'278'},
 {'87'},
 {'438'},
 {'215'},
 {'303'},
 {'69'},
 {'446'},
 {'205'},
 {'90'},
 {'136'},
 {'294'},
 {'40'},
 {'316'},
 {'138'},
 {'83'},
 {'2'},
 {'77'},
 {'64'},
 {'55'},
 {'4'},
 {'27'},
 {'148'},
 {'7'},
 {'218'},
 {'1'},
 {'1', '6'},
 {'3'},
 {'11', '3'},
 {'11', '3', '6'},
 {'3', '6'},
 {'11'},
 {'11', '6'},
 {'6'}]